Prima versione del modello con:
- epoche = 10
- batch size = 128
- learning reate = da 0.0002 a 0.0001
- frozen the weights of the first 50 layers

Questo progetto si basa su un modello di Resnet-50 (modelli che sono stati pre-allenati sul dataset ImageNet).


In [2]:
from models.CustomResNet50 import CustomResNet50

model = CustomResNet50()

In [11]:
import os
import torch
from torch import nn
from models.LwCoronNet import add_gaussian_noise
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

# Definizione dell'ottimizzatore e della funzione di perdita
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002)
H, W = 224, 224
BATCH_SIZE = 128

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.1)),
    transforms.Resize((H, W)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: add_gaussian_noise(x) if torch.rand(1).item() > 0.5 else x),
])
path_to_train = os.path.join(".", "Data", "train")
train_dataset = datasets.ImageFolder(path_to_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [12]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5144
    Root location: ./Data/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.9, 1.1), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Lambda()
           )

In [13]:
# Loop di addestramento
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:  # train_loader è il DataLoader dei dati di training
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


KeyboardInterrupt: 